In [58]:
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt 
import pandas as pd 
#import backtesting as bt
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas_ta as ta
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def BOLL(values, n, m):
    """
    Return Bollinger Bands for `values`, at
    each step taking into account `n` previous values
    and `m` standard deviations away from the mean.
    """
    sma = SMA(values, n)
    std = pd.Series(values).rolling(n).std()
    return sma, sma + m * std, sma - m * std

def Dochian(data, high_length, low_length):
    """
    Return Dochian Channel for `values`, at
    each step taking into account `n` previous values
    """
    return ta.donchian(data.High.s, data.Low.s, low_length, high_length, offset=1, talib=False)

def ATR(data, n):
    return np.array(ta.atr(high=data.High.s, low=data.Low.s, close=data.Close.s, length=n, talib=False))
    
    # tr = pd.Series(np.maximum(high - low, high - close, close - low))
    # return tr.rolling(n).mean()\

In [70]:
# Strategy



class turtleTrading(Strategy):
    upper_legnth = 20
    lower_length = 10
    atr_length = 20
    stop_loss_ratio = 11
    portion_ratio = 0.02
    
    def init(self):    
        #print(ATR(self.data, 20))
        self.atr = self.I(ATR, self.data, self.atr_length)
        self.donchian = self.I(Dochian, self.data, self.upper_legnth, self.lower_length)
        self.last_buy_price = 0
        self.last_buy_atr = 0
        self.buy(size=1)
        self.last_buy_price = self.data.Close[0]
        self.last_buy_atr = self.atr[0]
        
    def next(self):
        # If today's close exceed the donchian_upper_bound, buy, record price and ATR
        if crossover(self.data.Close, self.donchian[2]):
            portion = min((self.portion_ratio*self.data.Close[-1])/self.atr[-1], 1)
            self.buy(size=portion)
            self.last_buy_price = self.data.Close[-1]
            self.last_buy_atr = self.atr[-1]
        # If today's close is below the donchian_lower_bound, sell (end trade)
        elif crossover(self.donchian[0], self.data.Close):
            self.position.close() 
        # If price decresed by twice N at the last buy point, sell all (stop loss)
        elif self.position and self.data.Low[-1] < (self.last_buy_price - self.stop_loss_ratio * self.last_buy_atr):
            self.position.close()
            


In [ ]:
stock_code = "REGN" 
data_collecting_begaining_date = "2020-01-01"
data_collecting_ending_date = "2024-11-16"
period_of_collecting = "1d" 

def stock_price_collecting(code, datestart, dateend, period):
    data = yf.Ticker(code)
    prices = data.history(start=datestart, end=dateend, interval=period)
    return prices

data = stock_price_collecting(stock_code, data_collecting_begaining_date, data_collecting_ending_date, period_of_collecting)
data = pd.DataFrame(data)
train_data = data['2020-01-01':'2023-01-01']
test_data = data['2023-01-02':'2024-11-16']

bt = Backtest(train_data, turtleTrading, cash=10000)
stats = bt.optimize(
    upper_legnth=range(10, 100, 10),
    lower_length=range(5, 50, 5),
    atr_length=range(10, 40, 10),
    stop_loss_ratio=range(1, 5, 1),
    portion_ratio= [x/100 for x in range(1,5,1)],
    constraint=lambda p: p.upper_legnth > p.lower_length,
    maximize="Return [%]"
)
#stats = bt.run()

# Print optimized parameters
print("Optimized Parameters:")
print(f"Upper Length: {stats._strategy.upper_legnth}")
print(f"Lower Length: {stats._strategy.lower_length}")
print(f"ATR Length: {stats._strategy.atr_length}")
print(f"Stop Loss Ratio: {stats._strategy.stop_loss_ratio}")
print(f"Portion Ratio: {stats._strategy.portion_ratio}")

print(stats)
bt.plot()

In [ ]:
turtleTrading.upper_legnth = stats._strategy.upper_legnth
turtleTrading.lower_length = stats._strategy.lower_length
turtleTrading.atr_length = stats._strategy.atr_length
turtleTrading.stop_loss_ratio = stats._strategy.stop_loss_ratio
turtleTrading.portion_ratio = stats._strategy.portion_ratio

bt_test = Backtest(test_data, turtleTrading, cash=100000)
stats_test = bt_test.run()
print(stats_test)
bt_test.plot()

Start                     2023-01-03 00:00...
End                       2024-11-15 00:00...
Duration                    682 days 00:00:00
Exposure Time [%]                   40.677966
Equity Final [$]                 11302.428528
Equity Peak [$]                  11505.469055
Return [%]                          12.821207
Buy & Hold Return [%]                5.043934
Return (Ann.) [%]                    6.652571
Volatility (Ann.) [%]                9.428814
Sharpe Ratio                         0.705558
Sortino Ratio                        1.311267
Calmar Ratio                         0.662913
Max. Drawdown [%]                  -10.035356
Avg. Drawdown [%]                   -3.000895
Max. Drawdown Duration      256 days 00:00:00
Avg. Drawdown Duration       91 days 00:00:00
# Trades                                   23
Win Rate [%]                        43.478261
Best Trade [%]                      12.976929
Worst Trade [%]                      -7.79476
Avg. Trade [%]                    

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p10781', ...)

In [ ]:
from backtesting.backtesting import _Broker
from functools import partial

class turtleTradin_active():
    upper_legnth = 20
    lower_length = 10
    atr_length = 20
    stop_loss_ratio = 11
    portion_ratio = 0.02
    
    def init(self, data, *args, **kwargs):    
        self.data = data
        self.atr = self.I(ATR, self.data, self.atr_length)
        self.donchian = self.I(Dochian, self.data, self.upper_legnth, self.lower_length)
        self.last_buy_price = 0
        self.last_buy_atr = 0
        
    def I(self, func, *args, **kwargs):
        return partial(func, *args, **kwargs)
        
    def I
    def next(self):
        # If today's close exceed the donchian_upper_bound, buy, record price and ATR
        if crossover(self.data.Close, self.donchian[2]):
            portion = min((self.portion_ratio*self.data.Close[-1])/self.atr[-1], 1)
            assert_amount = round(portion * self.cash/self.data.Close[-1])
            ## self.buy(size=portion)
            print("buy at", self.data.Close[-1], "with amount", assert_amount)
            self.last_buy_price = self.data.Close[-1]
            self.last_buy_atr = self.atr[-1]
            
        # If today's close is below the donchian_lower_bound, sell (end trade)
        elif crossover(self.donchian[0], self.data.Close):
            #self.position.close() 
            print("close curent postion")
        # If price decresed by twice N at the last buy point, sell all (stop loss)
        elif self.position and self.data.Low[-1] < (self.last_buy_price - self.stop_loss_ratio * self.last_buy_atr):
            #self.position.close()
            print("close current postion")
        else:
            print("do nothing")
            
_broker = _Broker(
    data=data, cash=100_000, commission=0, margin=1.,
    trade_on_close=False, hedging=False,
    exclusive_orders=False, index=data.index,
)
active = turtleTradin_active(data=test_data, broker=_broker, params={})


